In [1]:
# unstandard libs:
# !pip install pikepdf

import time

# tkinter modules
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

# pikepdf and progressbar modules
import pikepdf
from tqdm import tqdm

# starting main window loop
window = tk.Tk()

In [2]:
# describe tkinter variables that we need
K = {'pdf_file_path' : tk.Variable(),
     'display_names' : tk.Variable()
     }

In [3]:
# describe window's parameters
window.resizable(0, 0)
window.title('UnlockPDF')
window.geometry('400x120')

window.columnconfigure(0, weight=2)
window.rowconfigure(0)

# a comand to close window in the end
def CloseWindow():
    window.quit()
    window.destroy()

# window's structure
frame_upper = ttk.Frame(window, width=20, height=20)
frame_upper.grid(column=0, row=0, sticky=tk.NSEW, padx=5, pady=5)

frame_upper.columnconfigure(0, weight=4)

frame_upper.rowconfigure(0, weight=1)
frame_upper.rowconfigure(1, weight=1)
frame_upper.rowconfigure(2, weight=10)

# main function that do everything after you pick files
def convertPDF(event=None):
    file_paths = filedialog.askopenfilenames(
        initialdir='',
        title="Open your PDF files",
        filetypes=(("PDF files", "*.pdf"),))
    files_count = len(file_paths)
    # after files are picked, they are converted in a cycle
    for single_file_path in file_paths:
        # there we open files and update info line in the window
        try:
            # then we add suffix to file name
            K['pdf_file_path'].set(single_file_path)
            K['display_names'].set(K['display_names'].get() + '\n' + single_file_path)
            upload_pdf_line.configure(text=K['pdf_file_path'].get())
            pdf = pikepdf.open(single_file_path, allow_overwriting_input=True)
            filename = str(K['pdf_file_path'].get()).split('/')[-1]
            new_filename = filename[0:-4]+'_unlocked'+'.pdf'
            
            # reconstruct the path for a new file
            slash = '/'
            new_path = slash.join(str(K['pdf_file_path'].get()).split('/')[0:-1])+slash+new_filename
            pdf.save(new_path)
            
            # optional wait period
            # time.sleep(0.5)
            print(new_path, ' is ready')
            window.update()      

        
        # as adapted from a guide here
        # https://thepythoncode.com/article/crack-pdf-file-password-in-python
        except pikepdf._core.PasswordError as e:
            # load password list
            passwords = [ line.strip() for line in open("wordlist.txt") ]

            # iterate over passwords
            for password in tqdm(passwords, "Decrypting PDF"):
                try:
                    # open PDF file
                    with pikepdf.open(single_file_path, password=password) as pdf:
                        # Password decrypted successfully, break out of the loop
                        print("[+] Password found:", password)
                        filename = str(K['pdf_file_path'].get()).split('/')[-1]
                        new_filename = filename[0:-4]+'_unlocked'+'.pdf'
                        
                        # reconstruct the path for a new file
                        slash = '/'
                        new_path = slash.join(str(K['pdf_file_path'].get()).split('/')[0:-1])+slash+new_filename
                        pdf.save(new_path)
                        
                        # optional wait period
                        # time.sleep(0.5)
                        print(new_path, ' is ready')
                        window.update()                        
                        break
                except pikepdf._core.PasswordError as e:
                    # wrong password, just continue in the loop
                    continue
            
        
        files_count -= 1
        if files_count == 0:
            #kill window
            print('Task completed')
            CloseWindow()

In [4]:
# that's the header of the window
upload_csv_label = ttk.Label(frame_upper, text="Pick your PDF files to unlock", width=150, justify='center')
upload_csv_label.grid(column=0, row=0, sticky=tk.NSEW, padx=0, pady=5, columnspan=1)

# that's the button
upload_csv_button = tk.Button(frame_upper, text='Browse', command=convertPDF, width=20)
upload_csv_button.grid(column=0, row=1, padx=0, pady=5, sticky=tk.NSEW)

# and that's a line to show a name of a last converted file
upload_pdf_line = ttk.Label(frame_upper, text=K['display_names'].get(), wraplength=300, borderwidth=1, relief='sunken', width=50, background='black', foreground='white')
upload_pdf_line.grid(column=0, row=2, padx=0, rowspan=10, pady=5, sticky=tk.NSEW)    

In [5]:
# here we initate this window
window.mainloop()

C:/Users/admin/Desktop/Savinykh_Prakticheskie_raboty_SMK1_24_unlocked.pdf  is ready
Task completed
